# Calculate MOMENTS (define specifict number of channels)

- Define the channel to colapse
- Define the range of velocity

__No Genering the masking intensities__


In [7]:
# Packages to calculate the moments
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import units as u
from astropy.wcs import WCS
from astropy import constants as const
from spectral_cube import SpectralCube
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import copy

In [15]:
#--------------- Define the parameters -------------------
# Define the path to the data cubes
file1 = 'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/RESULTS/FITS/CORRECT/CII/CII_N159_SPAT_RES.fits'
file2 = 'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/RESULTS/FITS/CORRECT/CF+/CF+_N159_sci_VELAXIS_inK_SPECT_SPAT_RES.fits'

# Read the data cubes
cube1 = SpectralCube.read(file1)
cube2 = SpectralCube.read(file2)
print(cube1.header)
print(cube2.header)
#------------- Define the parameters for the moments calculation -------------------

nchan = 3  # Number of channels to collapse
vel_min = 190 * u.km / u.s # Minimum velocity
vel_max = 260 * u.km / u.s # Maximum velocity
order = 0  # Order of the moment (0 for intensity, 1 for velocity, 2 for dispersion)
print('Parameters for the moments calculation:')
print(f'Number of channels: {nchan}')
print(f'Velocity range: {vel_min} to {vel_max}')

# ------------- Define the output file names ---------------------
output_file1 = f'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/CODES/TEST/moments_cube1_nchan{nchan}.fits'
output_file2 = f'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/CODES/TEST/moments_cube2_nchan{nchan}.fits'

# Diferent output to other quantitie of channel to
#output_file1 = 'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/RESULTS/FITS/TESTS/moments__cube1_nch5.fits'
#output_file2 = 'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/RESULTS/FITS/TESTS/moments__cube2_nch5.fits'

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    3                                                  NAXIS1  =                   70                                                  NAXIS2  =                   60                                                  NAXIS3  =                  300                                                  RESTFRQ =                  0.0                                                  VELREF  =                    0                                                  BMAJ    = 0.004444444444444444                                                  BMIN    = 0.004444444444444444                                                  BPA     =                  0.0                                                  BUNIT   = 'K       '           / Brightness unit in Kelvin                      BEAM    = 'Beam: BMAJ=15.999999999999996

In [16]:
# Obtain essential information from the cubes
print('--------------------')
print('Información del cubo 1:')
UNIT1 = cube1.header['CUNIT3']
CRVAL1 = cube1.header['CRVAL3'] * u.km/u.s
CRPIX1 = cube1.header['CRPIX3']
CDELT1 = cube1.header['CDELT3'] * u.km/u.s
print('UNIT:', UNIT1)
print('CRVAL:', CRVAL1)
print('CRPIX:', CRPIX1)
print('CRDELT:', CDELT1)

print('--------------------')
print('Información del cubo 2:')
UNIT2 = cube2.header['CUNIT3']
RESTFRQ2 = cube2.header['RESTFRQ'] * u.Hz
CRVAL2 = cube2.header['CRVAL3'] * u.km/u.s
CRPIX2 = cube2.header['CRPIX3']
CDELT2 = cube2.header['CDELT3'] * u.km/u.s
print('UNIT en cubo 2:', UNIT2)
print('RESTFRQ en cubo 2:', RESTFRQ2)
print('CRVAL en cubo 2:', CRVAL2)
print('CRPIX en cubo 2:', CRPIX2)
print('CRDELT en cubo 2:', CDELT2)

# Obtain the number of channel
# Obtener el número de canales en los cubos
nchan1 = cube1.shape[0]
nchan2 = cube2.shape[0]
print('--------------------')
print('Número de canales en cubo 1:', nchan1)
print('Número de canales en cubo 2:', nchan2)

# Obtain speed and frequency axes
index1 = np.arange(1, nchan1 + 1)
VEL1 = CRVAL1 + (index1 - CRPIX1) * CDELT1
print('--------------------')
print('Eje de velocidad del cubo 1 (ejemplo):', VEL1[:5])

index2 = np.arange(1, nchan2 + 1)
VEL2 = CRVAL2 + (index2 - CRPIX2) * CDELT2
print('Eje de velocidad del cubo 2 (ejemplo):', VEL2[:5])


--------------------
Información del cubo 1:
UNIT: km s-1
CRVAL: 187.0 km / s
CRPIX: 150.0
CRDELT: 1.0 km / s
--------------------
Información del cubo 2:
UNIT en cubo 2: km s-1
RESTFRQ en cubo 2: 102587476000.0 Hz
CRVAL en cubo 2: -129.00000000003 km / s
CRPIX en cubo 2: 1.0
CRDELT en cubo 2: 1.0 km / s
--------------------
Número de canales en cubo 1: 300
Número de canales en cubo 2: 729
--------------------
Eje de velocidad del cubo 1 (ejemplo): [38. 39. 40. 41. 42.] km / s
Eje de velocidad del cubo 2 (ejemplo): [-129. -128. -127. -126. -125.] km / s


#### Generate the definition to calculate moment

In [17]:
def calculate_moment(cube, vel_min, vel_max, nchan, order, output_file):
    # Cortar cubo en velocidad
    subcube = cube.with_spectral_unit(u.km/u.s)
    subcube = subcube.spectral_slab(vel_min, vel_max)
    
    # Número total de canales del subcubo
    n_total_channels = subcube.shape[0]
    n_blocks = n_total_channels // nchan  # omite canales restantes
    
    # Lista para almacenar los mapas de momento
    moment_maps = []
    velocity_centers = []

    # Loop por bloques de canales
    for i in range(n_blocks):
        start = i * nchan
        end = start + nchan
        block = subcube[start:end, :, :]
        
        # Calcular el mapa de momento
        moment_map = block.moment(order=order)
        moment_maps.append(moment_map.data)  # Guardar solo los datos (array numpy)
        
        # Calcular el valor promedio de velocidad para el bloque
        vel_start = subcube.spectral_axis[start].value
        vel_end = subcube.spectral_axis[end-1].value if end <= n_total_channels else subcube.spectral_axis[-1].value
        vel_center = (vel_start + vel_end) / 2.0
        velocity_centers.append(vel_center)
    
    # Convertir la lista de mapas de momento en un cubo 3D
    moment_cube = np.stack(moment_maps, axis=0)  # Apila los mapas en el eje 0
    
    # Copiar el encabezado del cubo original
    header = copy.deepcopy(cube.header)
    
    # Modificar el encabezado para reflejar el nuevo cubo
    header['NAXIS'] = 3  # Cubo 3D
    header['NAXIS3'] = n_blocks  # Número de bloques (mapas de momento)
    header['CTYPE3'] = 'VELO-LSR'  # Eje espectral (velocidad)
    header['CUNIT3'] = 'km s-1'  # Unidad de velocidad
    header['CRVAL3'] = velocity_centers[0]  # Valor de referencia (velocidad del primer bloque)
    header['CDELT3'] = nchan  # Incremento de velocidad
    header['CRPIX3'] = 1  # Píxel de referencia
    header['BUNIT'] = f'{cube.header.get("BUNIT", "")} km s^-1' if order == 0 else cube.header.get('BUNIT', '')  # Ajustar unidad para momento 0
    
    # Mantener claves espectrales relevantes
    if 'SPECSYS' in cube.header:
        header['SPECSYS'] = cube.header['SPECSYS']  # Sistema de referencia espectral
    if 'RESTFRQ' in cube.header:
        header['RESTFRQ'] = cube.header['RESTFRQ']  # Frecuencia de reposo (si aplica)
    
    # Agregar comentario sobre los rangos de velocidad
    for i, vel in enumerate(velocity_centers, 1):
        header[f'V_CENTER{i}'] = f'{vel:.2f} km/s'
    
    # Crear el objeto HDU con los datos y el encabezado
    hdu = fits.PrimaryHDU(data=moment_cube, header=header)
    
    # Guardar el cubo en un archivo FITS
    hdu.writeto(output_file, overwrite=True)
    
    return moment_maps, velocity_centers


In [18]:
# ----------------- Run the function for both cubes -------------------
moment_maps1, velocity_ranges1 = calculate_moment(cube1, vel_min, vel_max, nchan, order, output_file1)
moment_maps2, velocity_ranges2 = calculate_moment(cube2, vel_min, vel_max, nchan, order, output_file2)

# Imprimir los rangos de velocidad para cada bloque
print(f'Moment maps for cube1 saved to {output_file1}')
for i, (vel_min, vel_max) in enumerate(velocity_ranges1, 1):
    print(f'Cube1 - Block {i}: Velocity range {vel_min:.2f} to {vel_max:.2f} km/s')

print(f'Moment maps for cube2 saved to {output_file2}')
for i, (vel_min, vel_max) in enumerate(velocity_ranges2, 1):
    print(f'Cube2 - Block {i}: Velocity range {vel_min:.2f} to {vel_max:.2f} km/s')

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/CODES/TEST/moments_cube1_nchan3.fits'

In [14]:
# Verificar que los archivos FITS generados se pueden abrir con SpectralCube
try:
    test_cube1 = SpectralCube.read(output_file1)
    print(f"Successfully opened {output_file1} with SpectralCube")
    print(test_cube1.header)
except Exception as e:
    print(f"Failed to open {output_file1} with SpectralCube: {e}")

try:
    test_cube2 = SpectralCube.read(output_file2)
    print(f"Successfully opened {output_file2} with SpectralCube")
    print(test_cube2.header)
except Exception as e:
    print(f"Failed to open {output_file2} with SpectralCube: {e}")

Successfully opened C:/Users/macka/OneDrive/Documentos/Master/PYTHON_CODES/CODES/TEST/moments_cube1_nchan3.fits with SpectralCube
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    3                                                  NAXIS1  =                   70                                                  NAXIS2  =                   60                                                  NAXIS3  =                   10                                                  RESTFRQ =                  0.0                                                  VELREF  =                    0                                                  BMAJ    = 0.004444444444444444                                                  BMIN    = 0.004444444444444444                                                  BPA     =                  0.0                                        